In [33]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [34]:
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_SVR_Model_Hour.csv'

In [35]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')

In [36]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 368849 entries, 1 to 368849
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_MEDICAO_HORA         368849 non-null  object 
 1   PRECIPITACAO            368849 non-null  float64
 2   PRESSAO_ATMOSFERICA     368849 non-null  float64
 3   TEMPERATURA_DO_AR_C     368849 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  368849 non-null  float64
 5   VELOCIDADE_VENTO        368849 non-null  float64
 6   SK_PONTO                368849 non-null  int64  
 7   VL_MEDICAO              368849 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 25.3+ MB


In [37]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [38]:
def salvar_resultado(sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  data = [sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration]
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)   

In [39]:
def previsao_SVR(sk_ponto, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['VL_MEDICAO'].shift(n_step)  
    
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 6 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 6 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   

  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)   

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  salvar_resultado(sk_ponto, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, Duracao)



In [40]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    for n_time_steps in range(1,7): #predict with 1 to 6 past values of medition
        previsao_SVR(sk, df_ponto, n_time_steps)

forecast for sk_ponto =  5
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[5, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-08, 'kernel': 'rbf'}", 1, 10436.802317930482, 102.16066913411679, 63.41769524505776, 8.982583712549399, 63.14264440536499]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[5, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-08, 'kernel': 'rbf'}", 2, 10403.388019618546, 101.99700005205322, 61.713765817919466, 8.81638100695997, 65.64730191230774]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


...

['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[9, "{'C': 1255555, 'epsilon': 0.01, 'gamma': 1e-08, 'kernel': 'rbf'}", 4, 1135.3833447989732, 33.69544991239876, 31.566783669280397, 7.3405604674892695, 20.51500630378723]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[9, "{'C': 12550, 'epsilon': 0.0001, 'gamma': 1e-07, 'kernel': 'rbf'}", 5, 926.3853519195914, 30.436579175715384, 30.81933515441125, 6.084496882522892, 20.510406732559204]
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


['SK_PONTO', 'Best Params', 'N_Past_Vl', 'MSE', 'RMSE', 'MAE', 'MAPE', 'Duration']
[9, "{'C': 12550, 'epsilon': 0.1, 'gamma': 1e-08, 'kernel': 'rbf'}", 6, 306.9956551392954, 17.52129148034743, 15.679233894788183, 3.5254385657870215, 19.128125190734863]
